In [12]:
import pandas as pd
import numpy as np
import os

# Pfade
input_csv = "../data/immowelt_wohnungen_complete.csv"
output_csv = "../data/immowelt_daten_clean.csv"

# CSV laden
df = pd.read_csv(input_csv)

# Whitespace aus Spaltennamen entfernen
df.columns = df.columns.str.strip()

# Preis bereinigen
df["Preis"] = (
    df["Preis"]
    .astype(str)
    .str.replace(r"\s+", "", regex=True)
    .str.replace("€", "")
    .str.replace("/Monat", "", regex=False)
    .str.replace(",", ".")
    .str.replace(".", "", regex=False)
    .str.replace(r"[^\d\.]", "", regex=True)
    .replace("", np.nan)
    .astype(float)
)

# Wohnfläche bereinigen
df["Wohnfläche"] = (
    df["Wohnfläche"]
    .astype(str)
    .str.extract(r"([0-9,.]+)")
    .iloc[:, 0]
    .str.replace(",", ".")
    .astype(float)
)

# Zimmer bereinigen
df["Zimmer"] = (
    df["Zimmer"]
    .astype(str)
    .str.extract(r"([0-9,.]+)")
    .iloc[:, 0]
    .str.replace(",", ".")
    .astype(float)
)

# Preis pro qm
df["Preis pro qm in euro"] = df["Preis"] / df["Wohnfläche"]

# Ausstattung zählen (zählt "Ja")
features = ["Möbeliert", "Balkon/Terrasse/Garten", "Keller", "Aufzug", "Stellplatz"]
df["Ausstattungsgrad_neu"] = df[features].apply(lambda row: sum(row == "Ja"), axis=1)

# Gruppieren des Ausstattungsgrads in 3 Kategorien
def gruppiere_ausstattungsgrad(grad):
    if grad <= 1:
        return 0  # Niedrig
    elif grad <= 3:
        return 1  # Mittel
    else:
        return 2  # Hoch

df["Ausstattungsgrad_gruppe"] = df["Ausstattungsgrad_neu"].apply(gruppiere_ausstattungsgrad)

# Ausstattungs-Faktoren für Anpassung
AUSSTATTUNGS_FAKTOREN = {
    0: 0.90,  # niedrige Ausstattung -> "preiswerter" Faktor
    1: 1.00,  # mittlere Ausstattung -> neutral
    2: 1.10   # hohe Ausstattung -> höherer Faktor erlaubt höheren Preis
}

# Angepasster Preis pro qm (angepasst nach Ausstattungsgrad)
df["Preis_pro_qm_angepasst"] = df.apply(
    lambda row: row["Preis pro qm in euro"] / AUSSTATTUNGS_FAKTOREN[row["Ausstattungsgrad_gruppe"]],
    axis=1
)

# Quantile pro Bezirk berechnen (10%, 45%, 85%) basierend auf "angepasstem" Preis pro qm
bezirk_quantile = {}
for bezirk, gruppe in df.groupby("Bezirk"):
    q = gruppe["Preis_pro_qm_angepasst"].quantile([0.10, 0.45, 0.85]).to_dict()
    bezirk_quantile[bezirk] = {
        "q10": q.get(0.10, np.nan),
        "q45": q.get(0.45, np.nan),
        "q85": q.get(0.85, np.nan)
    }

# Fairness-Label basierend auf angepasstem Preis pro qm und Bezirk
def bewertung_label(row):
    bezirk = row["Bezirk"]
    preis_qm_adj = row["Preis_pro_qm_angepasst"]
    q = bezirk_quantile.get(bezirk)
    if not q or pd.isna(q["q10"]):
        return np.nan

    if preis_qm_adj <= q["q10"]:
        return "Scam-Verdacht"
    elif preis_qm_adj <= q["q45"]:
        return "Günstig"
    elif preis_qm_adj <= q["q85"]:
        return "Fair"
    else:
        return "Zu teuer"

df["Fairness_Label"] = df.apply(bewertung_label, axis=1)

# Nur relevante Spalten behalten und NaN-Zeilen entfernen
relevant = [
    "Preis", "Wohnfläche", "Zimmer", "Bezirk", "Ausstattungsgrad_gruppe",
    "Beschreibung", "Adresse", "Preis pro qm in euro", "Preis_pro_qm_angepasst", "Fairness_Label"
]
df_clean = df[relevant].dropna()

# Speichern
os.makedirs("../data", exist_ok=True)
df_clean.to_csv(output_csv, index=False)

print(f"Bereinigung + Labeling abgeschlossen. Datei gespeichert: {output_csv}")

df_clean.head()


✅ Bereinigung + Labeling abgeschlossen. Datei gespeichert: ../data/immowelt_daten_clean.csv


,Preis,Wohnfläche,Zimmer,Bezirk,Ausstattungsgrad_gruppe,Beschreibung,Adresse,Preis pro qm in euro,Preis_pro_qm_angepasst,Fairness_Label
0,610.0,47.0,2.0,Tempelhof-Schöneberg,1,TAUSCHWOHNUNG 2 Zimmer Wohnung in Tempelhof ge...,"Mariendorf,Berlin(12107)",12.978723,12.978723,Fair
1,2050.0,111.0,4.0,Spandau,2,"Halske Gärten - Moderne, lichtdurchflutete Woh...","Elsa-Neumann-Straße 53,Siemensstadt,Berlin(13629)",18.468468,16.789517,Günstig
2,1600.0,62.0,2.0,Mitte,1,Rückzugsort in Mitte: Esplanade Residence\n\nD...,"Bellevuestraße 1,Tiergarten,Berlin(10785)",25.806452,25.806452,Fair
3,880.0,80.0,3.0,Charlottenburg-Wilmersdorf,1,TAUSCHWOHNUNG Biete Dreizimmerwohnung und such...,"Schmargendorf,Berlin(14193)",11.000000,11.000000,Günstig
4,1750.0,64.0,2.0,Tempelhof-Schöneberg,1,Exklusive Stadtwohnung mit Blick auf die Apost...,"Else-Lasker-Schüler-Straße 4,Schöneberg,Berlin...",27.343750,27.343750,Zu teuer
